In [51]:
%reset -f

In [12]:
from bs4 import BeautifulSoup
from os import path
import os
import pandas as pd
from pandas import Series, DataFrame
import re
import xml.etree.ElementTree as et
import zipfile

directory_to_extract_to = r'C:/Users/balob/Downloads/DATA_IR21'
ir_files = r'C:/Users/balob/Downloads/new1/'
downloads = r'C:/Users/balob/Downloads/DATA_IR21'

In [13]:
# Распаковка zip архивов документов IR21 в папку downloads

def unzip_file(path_to_zip_file, name):
    zip_ref = zipfile.ZipFile(path_to_zip_file, 'r')
    zip_ref.extractall(directory_to_extract_to)
    zip_ref.close()
    
def find_txt_files_and_folders(path_name):
    file_names_all = os.listdir(path_name)
    files = [file for file in file_names_all if path.isfile(path.join(path_name, file)) and file.find('.zip') != -1]
    folders = [folder for folder in file_names_all if path.isdir(path.join(path_name, folder))]
    return files, folders

all_zip_files = []

for folder in find_txt_files_and_folders(ir_files)[1]:
    files = find_txt_files_and_folders(path.join(ir_files, folder))[0]
    for zip_file in files:
        unzip_file(path.join(path.join(ir_files, folder), zip_file), directory_to_extract_to)
    all_zip_files.append(files)

all_zip_files

[]

In [6]:
# Подготовка списка файлов, расположенных в downloads

def find_txt_files_and_folders(path_name):
    file_names_all = os.listdir(path_name)
    files = [file for file in file_names_all if path.isfile(path.join(path_name, file)) and file.find('.xml') != -1]
    return files

ir21_xml_files = find_txt_files_and_folders(downloads)
ir21_xml_files

[]

In [4]:
# Функция для поиска связки TAGIG - MCCMNC

def find_tadig_mccmnc(downloads, file_ir21):
    tree = et.parse(path.join(downloads, file_ir21))
    root = tree.getroot()
    ns = re.match(r'{.*}', root.tag).group(0)
    
    sender_tadig = root.find(f'{ns}RAEXIR21FileHeader').find(f'{ns}SenderTADIG').text
    organisation_name = root.find(f'{ns}OrganisationInfo').find(f'{ns}OrganisationName').text
    
    tadig_mccmnc_list = []
    
    for network in root.iter(f'{ns}Network'):
        tadig = network.find(f'{ns}TADIGCode').text
        for mccmnc in network.iter(f'{ns}CCITT_E212_NumberSeries'):
            mccmnc = ''.join([mccmnc.find(f'{ns}MCC').text, mccmnc.find(f'{ns}MNC').text])
            tadig_mccmnc_list.extend([[organisation_name, sender_tadig, tadig, mccmnc]])

    return tadig_mccmnc_list

In [5]:
# Загрузка связки TAGIG - MCCMNC в DataFrame

df = DataFrame(columns = ['NETWORK_NAME', 'SENDER_TADIG', 'TADIG', 'MCCMNC'])

for file_ir21 in ir21_xml_files:
    try:
        tadig_mccmnc_list = find_tadig_mccmnc(downloads, file_ir21)
    except:
        pass
    df = df.append(DataFrame(tadig_mccmnc_list, columns = ['NETWORK_NAME', 'SENDER_TADIG', 'TADIG', 'MCCMNC']))
df.reset_index(drop=True, inplace=True)

df.head(3)

,NETWORK_NAME,SENDER_TADIG,TADIG,MCCMNC


In [12]:
df.to_csv(path.join(downloads, 'tadig_mccmnc_ir21.csv'), index=False)

In [7]:
len(df.SENDER_TADIG.unique())

217

In [ ]:
downloads = 'C:/Users/balob/Downloads'
file_ir21 = 'IR21_INDAT_Bharti Airtel Limited_20180307_03.xml'

with open(join(downloads,file_ir21)) as inf:
    ir21 = inf.read()

xml_ir21 = BeautifulSoup(ir21,'xml')

msisdn_list = []
for MSISDN_NumberRanges in xml_ir21.findAll('MSISDN_NumberRanges'):
    for MSISDN_RangeData in MSISDN_NumberRanges.findAll('MSISDN_RangeData'):
        for NumberRange in  MSISDN_RangeData.findAll('NumberRange'):
            msisdn_list.append([NumberRange.CC.text + NumberRange.NDC.text, 'msisdn'])

nn_list = []
for MSISDN_NumberRanges in xml_ir21.findAll('GT_NumberRanges'):
    for MSISDN_RangeData in MSISDN_NumberRanges.findAll('RangeData'):
        for NumberRange in  MSISDN_RangeData.findAll('NumberRange'):
            nn_list.append([NumberRange.CC.text + NumberRange.NDC.text, 'nn'])
            
msrn_list = []
for MSISDN_NumberRanges in xml_ir21.findAll('MSRN_NumberRanges'):
    for NumberRange in  MSISDN_NumberRanges.findAll('NumberRange'):
        msrn_list.append([NumberRange.CC.text + NumberRange.NDC.text, 'msrn'])
        
e164_list = []
e164_list.extend(msisdn_list)
e164_list.extend(nn_list)
e164_list.extend(msrn_list)

for tadigs in xml_ir21.findAll('SenderTADIG'):
    tadig = tadigs.text
    print(tadig)
    
df = DataFrame(e164_list, columns = ['e164', 'type'])
df['tadig'] = tadig
df['tadig'] = 'IRLDF'

display(df)

pd.merge(df_np.loc[df_np.Operator == tadig, ['Operator', 'Prefix', 'Number/Range', 'SSN']],
         df[df.type == 'nn'], how='left', left_on=['Operator', 'Prefix'], right_on=['tadig', 'e164'])